In [ ]:
# def KS(data, variable, n, bins_var, good_label, ):
#     churn=[]
#     non_churn=[]
#     for i in bins_var:
#         good.append((telecom[(variable>=i[0]) 
#                               & (variable<= i[1])]['Churn']==good_label).sum())
#         bad.append((telecom[(variable>=i[0]) 
#                               & (variable <= i[1])]['Churn']!=good_label).sum())
#     woe=pd.DataFrame({"Bins": bins_var,'Good': good, 'Bad':bad})
    
#     woe['Cum_good']=woe['Good'].cumsum()
#     woe['Cum_bad']=woe['Bad'].cumsum()
#     woe['%Cum_good']=woe['Cum_good'].apply(lambda x: x/woe['Good'].sum())
#     woe['%Cum_bad']=woe['Cum_bad'].apply(lambda x: x/woe['Bad'].sum())
#     woe['WOE']=ln(woe['%Cum_good']-woe['%Cum_bad']
#     return woe

In [ ]:
def is_monotonic(x):
    dx = np.diff(x)
    return np.all(dx <= 0) or np.all(dx >= 0)


def bins(data,variable, n):
    bins=np.linspace(data[variable].min(), data[variable].max(),n+1)
    ls=[]
    for i in range(0,n):
        ls.append((bins[i],bins[i+1]))
    return pd.Series(ls)    

def weight_of_evidence(data, variable,target, bins_var,good_label):
    good=[]
    bad=[]
    for i in bins_var:
        good.append((data[(data[variable]>=i[0]) 
                              & (data[variable]<= i[1])][str(target)]==good_label).sum())
        bad.append((data[(data[variable]>=i[0]) 
                              & (data[variable] <= i[1])][str(target)]!=good_label).sum())
    woe=pd.DataFrame({"Bins": bins_var,'Good': good, 'Bad':bad})
    
    woe['WOE']=np.log(woe['Good'])-np.log(woe['Bad'])
    woe['Info_value']=woe['WOE']*((woe['Good']/woe['Good'].sum())-(woe['Bad']/woe['Bad'].sum()))
    print("Information Value for {} is {}".format(variable, woe['Info_value'].sum()))
    print('WOE is monotonic:', is_monotonic(woe['WOE']))
    return woe


# UDF for calculating vif value
def vif_cal(input_data, dependent_col):
    vif_df = pd.DataFrame( columns = ['Var', 'Vif'])
    x_vars=input_data.drop([dependent_col], axis=1)
    xvar_names=x_vars.columns
    for i in range(0,xvar_names.shape[0]):
        y=x_vars[xvar_names[i]] 
        x=x_vars[xvar_names.drop(xvar_names[i])]
        rsq=sm.OLS(y,x).fit().rsquared  
        vif=round(1/(1-rsq),2)
        vif_df.loc[i] = [xvar_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis=0, ascending=False, inplace=False)

In [1]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(6, 4))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return fpr, tpr, thresholds